In [107]:
import pandas as pd
from string import ascii_uppercase as alphabet
import pickle

# 1. Extracting tables from groups A-H

In [108]:
# URL of the Wikipedia page containing the tables
url = "https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup"

# Reading all tables from the specified URL
all_tables = pd.read_html(url)

In [109]:
all_tables

[      0     1     2
 0   Oct   NOV   Dec
 1   NaN    15   NaN
 2  2021  2022  2023,
   2022 كأس العالم لكرة القدم Kaʾs al-ʿālam li-kurat al-qadam 2022 Qatar 2022 2022 قطر  \
 0                                         Now is All                                    
 1                                 Tournament details                                    
 2                                       Host country                                    
 3                                              Dates                                    
 4                                              Teams                                    
 5                                           Venue(s)                                    
 6                                      ← 2018 2026 →                                    
 
   2022 كأس العالم لكرة القدم Kaʾs al-ʿālam li-kurat al-qadam 2022 Qatar 2022 2022 قطر.1  
 0                                         Now is All                                     
 1         

In [110]:
all_tables[12]
# all_tables[19]
# all_tables[26]

# all_tables[61]

,Pos,Teamvte,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Qatar (H),0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Ecuador,0,0,0,0,0,0,0,0,Advance to knockout stage
2,3,Senegal,0,0,0,0,0,0,0,0,NaN
3,4,Netherlands,0,0,0,0,0,0,0,0,NaN


In [111]:
all_tables = pd.read_html("https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup")

# Dictionary to store the extracted tables
dict_table = {}

# Looping through the tables and assigning them to dictionary keys
for letter, i in zip(alphabet, range(12, 68, 7)):
    # Extracting a table from the list of tables
    df = all_tables[i]

    # Renaming the second column to 'Team'
    df.rename(columns={df.columns[1] : 'Team'}, inplace=True)

    # Removing the 'Qualification' column
    df.pop('Qualification')

    # Assigning the DataFrame to a dictionary key
    dict_table[f'Group {letter}'] = df

In [112]:
dict_table['Group B']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,England,0,0,0,0,0,0,0,0
1,2,Iran,0,0,0,0,0,0,0,0
2,3,United States,0,0,0,0,0,0,0,0
3,4,Wales,0,0,0,0,0,0,0,0


In [113]:
# Open the file 'fifa2022_group_table' in write binary mode ('wb')
with open('fifa2022_group_table', 'wb') as output:
    # Pickle the dictionary 'dict_table' and write it to the opened file
    pickle.dump(dict_table, output)

# 2. Extracting football matches

In [ ]:
from bs4 import BeautifulSoup
import requests

# List of Fifa World Cup years
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974, 1978,
         1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014, 2018]

# Function to scrape World Cup match data for a given year
def get_matches(year):
    """
    This function scrapes match data (home team, score, away team) from Wikipedia for a specific World Cup year.
    
    Args:
    year (int): The World Cup year to scrape data from.

    Returns:
    pandas.DataFrame: A DataFrame containing scraped match data for the given year with an additional 'Year' column.
    """

    # Construct URL for the Wikipedia page of the specified year's FIFA World Cup
    url = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'

    # Send an HTTP GET request to retrieve the webpage content
    response = requests.get(url)
    content = response.text

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(content, 'lxml')

    # Find all elements containing match details
    matches = soup.find_all('div', class_='footballbox')

    # Initialize empty lists to store match details
    home = []
    score = []
    away = []

    # Loop through each match element and extract relevant details
    for match in matches:
        home.append(match.find('th', class_='fhome').get_text())
        score.append(match.find('th', class_='fscore').get_text())
        away.append(match.find('th', class_='faway').get_text())

    # Create a dictionary to store match details
    dict = {'Home' : home, 'Score' : score, 'Away' : away}

    # Create a DataFrame from the dictionary
    df_matches = pd.DataFrame(dict)

    # Add a column for the year of the FIFA World Cup
    df_matches['Year'] = year
    
    return df_matches

# Retrieve match data for each FIFA World Cup year using list comprehension
fifa = [get_matches(year) for year in years]

# Concatenate the DataFrames into a single DataFrame
df_fifa = pd.concat(fifa, ignore_index=True)

# Save the combined DataFrame to a CSV file
df_fifa.to_csv('all_fifa_worldcups_data.csv', index=False)

In [ ]:
# Counting the occurrences of each year in the 'Year' column of df_fifa_data
df_fifa['Year'].value_counts().sort_index()

## 2.1 Extracting FIFA 2022 fixtures

In [162]:
url = ("https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup")
    
response = requests.get(url)
content = response.text
soup = BeautifulSoup(content, 'lxml')

# Find all elements containing fixture details
fixtures = soup.find_all('div', class_='footballbox')

# Using list comprehension to extract data from fixtures
home = [fixture.find('th', class_='fhome').get_text() for fixture in fixtures]
score = [fixture.find('th', class_='fscore').get_text() for fixture in fixtures]
away = [fixture.find('th', class_='faway').get_text() for fixture in fixtures]

# Creating a dictionary from the lists
fixture_dict = {'Home': home, 'Score': score, 'Away': away}

# Creating a DataFrame from the dictionary
df_fixtures = pd.DataFrame(fixture_dict)

# Adding the year column
df_fixtures['Year'] = 2022

# Exporting the DataFrame to a CSV file
df_fixtures.to_csv('fifa2022_fixtures.csv', index=False)

## 2.2 Extracting missing data (1990)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Initialise Firefox WebDriver using the service
driver = webdriver.Firefox()
driver.maximize_window()

# List of Fifa World Cup years
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974, 1978,
         1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014, 2018]

def get_misssing_data(year):
    """
    This function retrieves match data (home team, score, away team) for a specific World Cup year 
    by scraping the Wikipedia page.

    Args:
    year (int): The World Cup year to scrape data from.

    Returns:
    pandas.DataFrame: A DataFrame containing scraped match data for the given year with an additional 'Year' column.
    """
    
    # Construct URL for the Wikipedia page of the specified year's FIFA World Cup
    url = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'

    # Open the desired website
    driver.get(url)

    # Extracting data for each match from the table
    matches = driver.find_elements(By.CSS_SELECTOR, 'tr[style="font-size:90%"]')

    # Initialize empty lists to store match details
    home = []
    score = []
    away = []

    # Loop through each match element and extract relevant details
    for match in matches:
        home.append(match.find_element(By.CSS_SELECTOR, 'td[align="right"] a').text)
        score.append(match.find_element(By.CSS_SELECTOR, 'td[align="center"]').text)
        away.append(match.find_element(By.CSS_SELECTOR, 'td:nth-child(3) a').text)

    # Create a dictionary to store match details
    dict = {'Home': home, 'Score': score, 'Away': away}
    
    # Create a DataFrame from the dictionary
    df_matches = pd.DataFrame(dict)

    # Add a column for the year of the FIFA World Cup
    df_matches['Year'] = year

    # Pausing execution for 2 seconds to allow time for the webpage to load
    time.sleep(2)
    
    return df_matches

# Retrieve match data for each FIFA World Cup year using list comprehension
fifa = [get_misssing_data(year) for year in years]
# fifa = get_misssing_data(1990)

# Closing the WebDriver after scraping
driver.quit()

# Concatenate the DataFrames into a single DataFrame
df_fifa = pd.concat(fifa, ignore_index=True)

# Save the combined DataFrame to a CSV file
df_fifa.to_csv("fifa_worldcup_missing_data.csv", index=False)

# 3. Data cleaning and transformation

In [163]:
df_fifa_data = pd.read_csv('all_fifa_worldcups_data.csv')
df_fixture = pd.read_csv('fifa2022_fixtures.csv')
df_missing_data = pd.read_csv('fifa_worldcup_missing_data.csv')

## 3.1 Cleaning fixtures and missing data

In [164]:
# Removing leading and trailing whitespace from the 'Home' and 'Away' columns
# Using apply() method to apply the strip() method to each element in the specified columns
df_fixture[['Home', 'Away']] = df_fixture[['Home', 'Away']].apply(lambda x: x.str.strip())
df_missing_data[['Home', 'Away']] = df_missing_data[['Home', 'Away']].apply(lambda x: x.str.strip())

In [165]:
df_fixture.isnull().sum()

Home     0
Score    0
Away     0
Year     0
dtype: int64

In [166]:
df_missing_data.isnull().sum()

Home     0
Score    0
Away     0
Year     0
dtype: int64

## 3.2 Merging Fifa data and missing data

In [167]:
fifa_data_df = pd.concat([df_fifa_data, df_missing_data], ignore_index=True)
fifa_data_df.drop_duplicates(inplace=True)
fifa_data_df.sort_values('Year', inplace=True)
fifa_data_df

,Home,Score,Away,Year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
829,Serbia,0–2,Brazil,2018
828,Serbia,1–2,Switzerland,2018
827,Brazil,2–0,Costa Rica,2018
825,Costa Rica,0–1,Serbia,2018


## 3.3 Cleaning merged data

In [168]:
# Iterating over each column to find and print the unique values
for column in fifa_data_df.columns:
    # Getting the unique values for the current column
    unique_values = fifa_data_df[column].unique()
    
    # Displaying the column name and its unique values
    display(f"Unique values in {column}: {unique_values}")

"Unique values in Home: ['France\\xa0' 'Uruguay\\xa0' 'Argentina\\xa0' 'Paraguay\\xa0'\n 'United States\\xa0' 'Romania\\xa0' 'Yugoslavia\\xa0' 'Chile\\xa0'\n 'Brazil\\xa0' 'Italy\\xa0' 'Germany\\xa0' 'Czechoslovakia\\xa0' 'Austria\\xa0'\n 'Sweden\\xa0' 'Switzerland\\xa0' 'Hungary\\xa0' 'Spain\\xa0' 'Cuba\\xa0'\n 'England\\xa0' 'West Germany\\xa0' 'Turkey\\xa0' 'Soviet Union\\xa0'\n 'Wales\\xa0' 'Northern Ireland\\xa0' 'Mexico\\xa0' 'North Korea\\xa0'\n 'Portugal\\xa0' 'Bulgaria\\xa0' 'Peru\\xa0' 'Belgium\\xa0' 'Poland\\xa0'\n 'Netherlands\\xa0' 'East Germany\\xa0' 'Haiti\\xa0' 'Australia\\xa0'\n 'Zaire\\xa0' 'Scotland\\xa0' 'Tunisia\\xa0' 'Honduras\\xa0' 'Algeria\\xa0'\n 'Denmark\\xa0' 'Morocco\\xa0' 'South Korea\\xa0' 'Iraq\\xa0' 'Canada\\xa0'\n 'Cameroon' 'Costa Rica' 'Brazil' 'Argentina' 'Austria' 'Italy'\n 'United States' 'Sweden' 'Soviet Union' 'South Korea'\n 'United Arab Emirates' 'Republic of Ireland' 'England' 'Netherlands'\n 'Belgium' 'Uruguay' 'Yugoslavia' 'West Germany' 'Ca

"Unique values in Score: ['4–1' '4–2' '6–1' '1–0' '3–0' '4–0' '3–1' '2–1' '6–3' '1–1 (a.e.t.)'\n '2–1 (a.e.t.)' '3–2' '3–2 (a.e.t.)' '7–1' '5–2' '8–0' '5–1' '2–0'\n '3–0 (a.e.t.)' '6–5 (a.e.t.)' '3–3 (a.e.t.)' 'w/o' '6–0' '2–2' '7–5'\n '4–2 (a.e.t.)' '4–4 (a.e.t.)' '7–0' '5–0' '8–3' '7–2' '9–0' '0–0' '1–3'\n '7–3' '3–3' '1–1' '4–4' '5–3' '0–1 (a.e.t.)' '4–3 (a.e.t.)' '0–1' '1–2'\n '0–2' '0–7' '0–3' '1–4' '1–5' '3–1 (a.e.t.)' '10–1' '3–4 (a.e.t.)'\n '0–0 (a.e.t.)' '2–4 (a.e.t.)' '2–3' '0–4' '2–3 (a.e.t.)' '1–0 (a.e.t.)'\n '1–2 (a.e.t.)' '2–2 (a.e.t.)' '1–0 (a.e.t./g.g.)' '2–1 (a.e.t./g.g.)'\n '1–2 (a.e.t./g.g.)' '0–1 (a.e.t./g.g.)' '2–5' '0–2 (a.e.t.)' '1–7' '2–4'\n '4–3']"

"Unique values in Away: ['\\xa0Mexico' '\\xa0Argentina' ' Yugoslavia' '\\xa0United States'\n '\\xa0Belgium' '\\xa0Paraguay' '\\xa0Romania' '\\xa0Peru' '\\xa0Bolivia'\n '\\xa0Brazil' '\\xa0Chile' '\\xa0France' '\\xa0Spain' '\\xa0Czechoslovakia'\n '\\xa0Austria' '\\xa0\\xa0Switzerland' '\\xa0Sweden' '\\xa0Hungary'\n '\\xa0Germany' '\\xa0Netherlands' '\\xa0Egypt' '\\xa0Cuba' '\\xa0Poland'\n '\\xa0Norway' '\\xa0Dutch East Indies' '\\xa0Italy' '\\xa0England'\n '\\xa0Yugoslavia' '\\xa0Uruguay' '\\xa0Scotland' '\\xa0Turkey'\n '\\xa0West Germany' '\\xa0South Korea' '\\xa0Soviet Union'\n '\\xa0Northern Ireland' '\\xa0Wales' '\\xa0Bulgaria' '\\xa0Colombia'\n '\\xa0North Korea' '\\xa0Portugal' '\\xa0Morocco' '\\xa0El Salvador'\n '\\xa0Israel' '\\xa0East Germany' '\\xa0Haiti' '\\xa0Australia' '\\xa0Zaire'\n '\\xa0Iran' '\\xa0Tunisia' '\\xa0New Zealand' '\\xa0Kuwait' '\\xa0Honduras'\n '\\xa0Cameroon' '\\xa0Algeria' '\\xa0Denmark' '\\xa0Iraq' '\\xa0Canada'\n 'Romania' 'Scotland' 'Sweden' 'Soviet Uni

'Unique values in Year: [1930 1934 1938 1950 1954 1958 1962 1966 1970 1974 1978 1982 1986 1990\n 1994 1998 2002 2006 2010 2014 2018]'

In [170]:
# Removing leading and trailing whitespace from the 'Home' and 'Away' columns
fifa_data_df[['Home', 'Away']] = fifa_data_df[['Home', 'Away']].apply(lambda x: x.str.strip())

In [171]:
# Filtering "fifa_data_df" to find rows where the value in the "Score" column is 'w/o'
rows_with_wo = fifa_data_df[fifa_data_df['Score'] == 'w/o']
rows_with_wo

,Home,Score,Away,Year
37,Sweden,w/o,Austria,1938


Sweden were awarded a walkover as Austria were unable to compete because of the Austrian Anschluss in March 1938.

In [172]:
# Finding the index of "rows_with_wo"
row_to_drop = rows_with_wo.index

# Dropping the rows with the identified indices
fifa_data_df.drop(row_to_drop, inplace=True)

In [173]:
# Replacing parentheses and their contents with an empty string
# fifa_data_df['Score'] = fifa_data_df['Score'].str.replace(r'\(.*?\)', '', regex=True)

In [174]:
# Checking if the 'Score' column contains any characters other than digits and the dash symbol '–'
fifa_data_df['Score'] = fifa_data_df['Score'].str.replace('[^\d–]', '', regex=True)

In [175]:
# Stripping leading and trailing whitespaces
fifa_data_df['Score'] = fifa_data_df['Score'].str.strip()

In [176]:
fifa_data_df[['HomeGoals', 'AwayGoals']] = fifa_data_df['Score'].str.split('–', expand=True)
fifa_data_df

,Home,Score,Away,Year,HomeGoals,AwayGoals
0,France,4–1,Mexico,1930,4,1
17,Uruguay,4–2,Argentina,1930,4,2
16,Uruguay,6–1,Yugoslavia,1930,6,1
15,Argentina,6–1,United States,1930,6,1
14,Paraguay,1–0,Belgium,1930,1,0
...,...,...,...,...,...,...
829,Serbia,0–2,Brazil,2018,0,2
828,Serbia,1–2,Switzerland,2018,1,2
827,Brazil,2–0,Costa Rica,2018,2,0
825,Costa Rica,0–1,Serbia,2018,0,1


In [177]:
# Deleting the 'Score' column from the DataFrame
fifa_data_df.drop('Score', axis=1, inplace=True)

In [178]:
# Renaming columns
fifa_data_df.rename(columns={'Home' : 'HomeTeam', 'Away' : 'AwayTeam'}, inplace=True)

# Converting columns to integer type
fifa_data_df = fifa_data_df.astype({'HomeGoals' : int, 'AwayGoals' : int})

# Creating new column "TotalGoals"
fifa_data_df['TotalGoals'] = fifa_data_df['HomeGoals'] + fifa_data_df['AwayGoals']

## 3.4 Exporting cleaned data

In [179]:
fifa_data_df.to_csv('cleaned_all_fifa_worldcups_data', index=False)
df_fixture.to_csv('cleaned_fifa2022_fixtures.csv', index=False)

## 3.5 Verification

In [180]:
# Counting the occurrences of each year in the 'Year' column of df_
fifa_data_df['Year'].value_counts().sort_index()

Year
1930    18
1934    17
1938    18
1950    22
1954    26
1958    35
1962    32
1966    32
1970    32
1974    38
1978    38
1982    52
1986    52
1990    52
1994    52
1998    64
2002    64
2006    64
2010    64
2014    64
2018    64
Name: count, dtype: int64

# 4. Prediction

In [235]:
from scipy.stats import poisson

In [236]:
fifa2022_grpTab = pickle.load(open('fifa2022_group_table','rb'))

## 4.1 Calculating Team Strength

In [237]:
# Extracting relevant columns for home and away teams
df_home = fifa_data_df[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = fifa_data_df[['AwayTeam', 'HomeGoals', 'AwayGoals']]

# Renaming columns for home and away teams
df_home = df_home.rename(columns={'HomeTeam': 'Team', 'HomeGoals': 'GoalsFor', 'AwayGoals': 'GoalsAgainst'})
df_away = df_away.rename(columns={'AwayTeam': 'Team', 'HomeGoals': 'GoalsAgainst', 'AwayGoals': 'GoalsFor'})

# Concatenating home and away team dataFrames and calculating mean 'goals for and against' for each team
df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsFor,GoalsAgainst
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


## 4.2 Function to predict match points

In [238]:
def predict_points(home, away):
    """
    Predicts points for home and away teams based on Poisson distribution

    Args:
    home (str): Name of the home team
    away (str): Name of the away team

    Returns:
      tuple: Tuple containing predicted points for home and away teams (float, float)
          - The first element is the predicted points for the home team.
          - The second element is the predicted points for the away team.
          - If either team is not found in the team strength data, returns (0, 0).
    """
    
    # Checking if both teams are present in the team strength DataFrame
    if home in df_team_strength.index and away in df_team_strength.index:
        # Calculating lambda values for Poisson distribution based on team strengths (attack and defence strengths)
        lamb_home = df_team_strength.at[home,'GoalsFor'] * df_team_strength.at[away,'GoalsAgainst']
        lamb_away = df_team_strength.at[away,'GoalsFor'] * df_team_strength.at[home,'GoalsAgainst']
        
        # Initialising probabilities for home win, away win, and draw (all set to 0 initially)
        prob_home, prob_away, prob_draw = 0, 0, 0
        
        # Looping through possible number of goals scored by home and away teams (0 to 10 goals each)
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                # Calculating the probability of the current goal combination using Poisson distribution
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                
                # Determining the outcome (home win, away win, or draw) based on the goal combination
                if x == y:
                    prob_draw += p # Add probability to draw
                elif x > y:
                    prob_home += p # Add probability to home win
                else:
                    prob_away += p # Add probability to away win

        # Calculating points based on win (3 points) and draw (1 point)
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        
        # Return the predicted points for home and away teams
        return (home, points_home), (away, points_away)
    else:
        # Return 0 points for both teams if either home or away team is not present in the team strength DataFrame
        return (home, 0), (away, 0)

In [239]:
# Testing function
print(predict_points('England', 'United States'))
print(predict_points('Nigeria', 'Ghana'))

# Qatar (H) made her debut in the FIFA World Cup in 2022
print(predict_points('Qatar (H)', 'Ecuador')) 

(('England', 2.2356147635326007), ('United States', 0.5922397535606193))
(('Nigeria', 1.321294620797116), ('Ghana', 1.4363162056734256))
(('Qatar (H)', 0), ('Ecuador', 0))


Based on the prediction model, if England plays the United States:
* England has a higher chance of winning and is predicted to earn around 2.23 points out of a possible 3.
* The United States is predicted to get fewer points (around 0.59), reflecting a lower chance of winning.

In football, a win typically awards 3 points, so these predictions suggest England is more likely to win and get the full 3 points, while the United States is less likely to win and might get a point for a draw.

# 5. Predicting the World Cup

## 5.1 Group stage

In [240]:
# Splitting fixtures into group stage, knockout, quarter, semi and final
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [241]:
# Looping through each group in the dictionary
for group in dict_table:
    # Getting the list of teams in the current group
    teams_in_group = dict_table[group]['Team'].values
    
    # Filtering the fixture DataFrame to include only matches involving teams in the current group
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['Home'].isin(teams_in_group)]
    
    # Looping through each fixture in the filtered fixture DataFrame
    for index, row in df_fixture_group_6.iterrows():
        # Getting the home and away teams for the current fixture
        home, away = row['Home'], row['Away']
        
        # Predicting points for the home and away teams in the fixture
        points_home, points_away = predict_points(home, away)
        
        # Updating the points for the home and away teams in the group DataFrame
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += int(points_home[1])
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += int(points_away[1])

    # Sorting the teams in the group DataFrame based on points in descending order
    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    
    # Retaining only the 'Team' and 'Pts' columns in the group DataFrame
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    
    # Rounding off the points to the nearest integer
    
    dict_table[group] = dict_table[group].round(0)


In [242]:
dict_table['Group A']

,Team,Pts
0,Netherlands,10
1,Ecuador,5
2,Senegal,5
3,Qatar (H),0


## 5.2 Knockout stage

In [243]:
df_fixture_knockout

,Home,Score,Away,Year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [244]:
# Iterating through each group in the dictionary
for group in dict_table:
    # Getting the winner and runner-up of the current group
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    
    # Replacing placeholders in the knockout fixture DataFrame with actual group winners and runners-up
    df_fixture_knockout.replace({f'Winners {group}': group_winner, f'Runners-up {group}': runners_up}, inplace=True)

# Adding a column named 'Winner' initialised with placeholder '?' to the knockout fixture DataFrame
df_fixture_knockout['Winner'] = '?'

# Displaying the updated knockout fixture DataFrame
df_fixture_knockout

,Home,Score,Away,Year,Winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Poland,2022,?
51,England,Match 51,Ecuador,2022,?
52,Germany,Match 53,Belgium,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Croatia,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [245]:
# Function to determine the winner of each fixture based on predicted points
def get_winner(df_fixture_updated):
    # Looking through each fixture in the updated fixture DataFrame
    for index, row in df_fixture_updated.iterrows():
        # Getting the home and away teams for the current fixture
        home, away = row['Home'], row['Away']
        
        # Predicting points for the home and away teams in the fixture
        points_home, points_away = predict_points(home, away)
        
        # Determining the winner based on points scored by each team
        if points_home > points_away:
            Winner = home
        else:
            Winner = away
        
        # Assigning the winner to the 'winner' column in the fixture DataFrame
        df_fixture_updated.loc[index, 'Winner'] = Winner
    
    # Returning the updated fixture DataFrame with the winner information
    return df_fixture_updated

In [246]:
get_winner(df_fixture_knockout)

,Home,Score,Away,Year,Winner
48,Netherlands,Match 49,Wales,2022,Wales
49,Argentina,Match 50,Denmark,2022,Denmark
50,France,Match 52,Poland,2022,Poland
51,England,Match 51,Ecuador,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Uruguay
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Switzerland


## 5.3 Quarter final

In [247]:
# Function to update the knockout table for the next round based on the results of the current round
def update_table(df_fixture_round_1, df_fixture_round_2):
    # Looping through each fixture in the first round DataFrame
    for index, row in df_fixture_round_1.iterrows():
        # Getting the winner and match identifier for the current fixture
        winner = df_fixture_round_1.loc[index, 'Winner']
        match = df_fixture_round_1.loc[index, 'Score']
        
        # Replacing placeholders in the second round fixture DataFrame with the winner of the current match
        df_fixture_round_2.replace({f'Winners {match}': winner}, inplace=True)
    
    # Adding a column named 'winner' initialized with placeholder '?' to the second round fixture DataFrame
    df_fixture_round_2['Winner'] = '?'
    
    # Returning the updated second round fixture DataFrame
    return df_fixture_round_2

In [248]:
update_table(df_fixture_knockout, df_fixture_quarter)

,Home,Score,Away,Year,Winner
56,Germany,Match 58,Uruguay,2022,?
57,Wales,Match 57,Denmark,2022,?
58,Spain,Match 60,Switzerland,2022,?
59,England,Match 59,Poland,2022,?


In [249]:
get_winner(df_fixture_quarter)

,Home,Score,Away,Year,Winner
56,Germany,Match 58,Uruguay,2022,Uruguay
57,Wales,Match 57,Denmark,2022,Wales
58,Spain,Match 60,Switzerland,2022,Switzerland
59,England,Match 59,Poland,2022,Poland


## 5.4 Semi final

In [250]:
update_table(df_fixture_quarter, df_fixture_semi)

,Home,Score,Away,Year,Winner
60,Wales,Match 61,Uruguay,2022,?
61,Poland,Match 62,Switzerland,2022,?


In [251]:
get_winner(df_fixture_semi)

,Home,Score,Away,Year,Winner
60,Wales,Match 61,Uruguay,2022,Wales
61,Poland,Match 62,Switzerland,2022,Switzerland


## 5.5 Final

In [252]:
update_table(df_fixture_semi, df_fixture_final)

,Home,Score,Away,Year,Winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Wales,Match 64,Switzerland,2022,?


In [253]:
get_winner(df_fixture_final)

,Home,Score,Away,Year,Winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Wales,Match 64,Switzerland,2022,Wales
